# Лабораторная работа: Fine-tuning EfficientNet-B0

**Цель работы:** обучить модель EfficientNet-B0 для классификации изображений трёх классов: `mouse`, `keyboard`, `soundcard`.

В ноутбуке показаны: подготовка данных, обучение, валидация, сохранение модели и экспорт в ONNX.


## 1. Импорт библиотек и настройка окружения

Запустите эту ячейку для импорта необходимых библиотек.

In [ ]:

# Ячейка: импорт библиотек
import os, sys, random, json, time
from pathlib import Path
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

import torch
import torch.nn as nn
from torch.utils.data import DataLoader
import timm

# Для датасета и аугментаций - используем torchvision и albumentations
from torchvision.datasets import ImageFolder
from torchvision import transforms as T
import albumentations as A
from albumentations.pytorch import ToTensorV2

print("torch:", torch.__version__)
print("timm:", timm.__version__)
print("albumentations:", A.__version__)


## 2. Параметры и пути

Редактируйте параметры по необходимости. Данные должны лежать в `data/raw/train` и `data/raw/val`. Модель будет сохранена в `models/best_efficientnet_b0.pth`. 

In [ ]:

# Настройки
ROOT = Path(".").resolve()
DATA_DIR = ROOT / "data" / "raw"
TRAIN_DIR = DATA_DIR / "train"
VAL_DIR = DATA_DIR / "val"
OUT_DIR = ROOT / "models"
OUT_DIR.mkdir(parents=True, exist_ok=True)

# Гиперпараметры
IMAGE_SIZE = 224
BATCH_SIZE = 16
EPOCHS = 12
LR = 1e-4
WEIGHT_DECAY = 1e-4
MODEL_NAME = "efficientnet_b0"
DEVICE = "cuda" if torch.cuda.is_available() else "cpu"
SEED = 42

print("DATA_DIR:", DATA_DIR)
print("DEVICE:", DEVICE)


## 3. Для воспроизводимости и вспомогательные функции

In [ ]:

# Воспроизводимость
def set_seed(seed:int=42):
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    if torch.cuda.is_available():
        torch.cuda.manual_seed_all(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False

set_seed(SEED)

# Визуализация примеров
def show_images_grid(paths, ncols=4, figsize=(12,8)):
    n = len(paths)
    nrows = (n + ncols - 1) // ncols
    plt.figure(figsize=figsize)
    for i, p in enumerate(paths):
        ax = plt.subplot(nrows, ncols, i+1)
        img = plt.imread(p)
        plt.imshow(img)
        plt.axis('off')


## 4. Загрузка и анализ данных

Проверим структуру директорий и классы.

In [ ]:

# Списки классов и примеры
assert TRAIN_DIR.exists(), f"Не найдена папка {TRAIN_DIR}"
assert VAL_DIR.exists(), f"Не найдена папка {VAL_DIR}"

train_classes = sorted([d.name for d in TRAIN_DIR.iterdir() if d.is_dir()])
val_classes = sorted([d.name for d in VAL_DIR.iterdir() if d.is_dir()])
print("Train classes:", train_classes)
print("Val classes:", val_classes)

# примеры файлов
sample_paths = []
for cls in train_classes[:3]:
    p = TRAIN_DIR / cls
    files = list(p.glob("*.jpg")) + list(p.glob("*.jpeg")) + list(p.glob("*.png"))
    sample_paths.extend(files[:4])

sample_paths = sample_paths[:12]
len(sample_paths), sample_paths[:3]


In [ ]:

# Покажем примеры (если есть)
if len(sample_paths)>0:
    show_images_grid(sample_paths, ncols=4)
else:
    print("Нет изображений для предпросмотра в тренировочной папке.")


## 5. Трансформации и DataLoader

Для тренировочного набора используем аугментации, для валидации — простое изменение размера и нормализацию.

In [ ]:

# Albumentations -> torchvision compatible wrapper
def get_albumentations_train(image_size=IMAGE_SIZE):
    return A.Compose([
        A.RandomResizedCrop(image_size, image_size, scale=(0.8,1.0)),
        A.HorizontalFlip(p=0.5),
        A.Rotate(limit=20, p=0.3),
        A.ColorJitter(0.2,0.2,0.2, p=0.3),
        A.Normalize(mean=(0.485,0.456,0.406), std=(0.229,0.224,0.225)),
        ToTensorV2(),
    ])

def get_albumentations_val(image_size=IMAGE_SIZE):
    return A.Compose([
        A.Resize(image_size, image_size),
        A.Normalize(mean=(0.485,0.456,0.406), std=(0.229,0.224,0.225)),
        ToTensorV2(),
    ])

# Wrapper dataset to apply albumentations on PIL images from ImageFolder
from torchvision.datasets import ImageFolder
class AlbumentationsImageFolder(ImageFolder):
    def __init__(self, root, transform=None, alb_transform=None):
        super().__init__(root, transform=transform)
        self.alb_transform = alb_transform
    def __getitem__(self, index):
        path, target = self.samples[index]
        # PIL open
        img = plt.imread(path)
        # If image has alpha channel, drop it
        if img.ndim == 2:
            img = np.stack([img]*3, axis=-1)
        if img.shape[2] == 4:
            img = img[..., :3]
        if self.alb_transform:
            augmented = self.alb_transform(image=img)
            img = augmented['image']
        return img, target

train_alb = AlbumentationsImageFolder(str(TRAIN_DIR), alb_transform=get_albumentations_train())
val_alb = AlbumentationsImageFolder(str(VAL_DIR), alb_transform=get_albumentations_val())

print("Samples train:", len(train_alb), " samples val:", len(val_alb))
class_names = train_alb.classes
print("Class names:", class_names)

train_loader = DataLoader(train_alb, batch_size=BATCH_SIZE, shuffle=True, num_workers=2)
val_loader = DataLoader(val_alb, batch_size=BATCH_SIZE, shuffle=False, num_workers=2)


## 6. Создание модели EfficientNet-B0

Используем `timm` для загрузки предобученной архитетуры и подгонки под количество классов.

In [ ]:

model = timm.create_model(MODEL_NAME, pretrained=True, num_classes=len(class_names))
model.to(DEVICE)
print(model)


## 7. Обучение модели

Цикл обучения с сохранением лучшей модели по валидационной точности.

In [ ]:

criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.AdamW(model.parameters(), lr=LR, weight_decay=WEIGHT_DECAY)
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=5, gamma=0.1)

best_val_acc = 0.0
history = {'train_loss':[], 'val_loss':[], 'val_acc':[]}

for epoch in range(1, EPOCHS+1):
    model.train()
    running_loss = 0.0
    count = 0
    for imgs, labels in train_loader:
        imgs = imgs.to(DEVICE)
        labels = labels.to(DEVICE)
        optimizer.zero_grad()
        outputs = model(imgs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        running_loss += loss.item() * imgs.size(0)
        count += imgs.size(0)
    train_loss = running_loss / count
    history['train_loss'].append(train_loss)

    # Validation
    model.eval()
    v_loss = 0.0
    v_count = 0
    correct = 0
    total = 0
    all_preds = []
    all_labels = []
    with torch.no_grad():
        for imgs, labels in val_loader:
            imgs = imgs.to(DEVICE)
            labels = labels.to(DEVICE)
            outputs = model(imgs)
            loss = criterion(outputs, labels)
            v_loss += loss.item() * imgs.size(0)
            v_count += imgs.size(0)
            preds = outputs.argmax(dim=1)
            correct += (preds == labels).sum().item()
            total += labels.size(0)
            all_preds.extend(preds.cpu().numpy().tolist())
            all_labels.extend(labels.cpu().numpy().tolist())
    val_loss = v_loss / v_count
    val_acc = correct / total
    history['val_loss'].append(val_loss)
    history['val_acc'].append(val_acc)

    print(f"Epoch {epoch}/{EPOCHS} - train_loss: {train_loss:.4f}  val_loss: {val_loss:.4f}  val_acc: {val_acc:.4f}")

    # Save best
    if val_acc > best_val_acc:
        best_val_acc = val_acc
        torch.save(model.state_dict(), OUT_DIR / "best_efficientnet_b0.pth")
        print("Saved best model.")
    scheduler.step()

# Сохраним историю
with open(OUT_DIR / "history_efficientnet_b0.json", "w") as f:
    json.dump(history, f, indent=2)


## 8. Построение графиков и матрицы ошибок

In [ ]:

# Графики loss/acc
hist = history
plt.figure(figsize=(10,4))
plt.subplot(1,2,1)
plt.plot(hist['train_loss'], label='train_loss')
plt.plot(hist['val_loss'], label='val_loss')
plt.legend(); plt.title('Loss')

plt.subplot(1,2,2)
plt.plot(hist['val_acc'], label='val_acc')
plt.legend(); plt.title('Val Accuracy')
plt.show()


In [ ]:

# Confusion matrix and classification report
from sklearn.metrics import confusion_matrix, classification_report
import seaborn as sns

# Load best model for evaluation
best = OUT_DIR / "best_efficientnet_b0.pth"
if best.exists():
    model.load_state_dict(torch.load(best, map_location=DEVICE))
    model.to(DEVICE)
    model.eval()

    all_preds, all_labels = [], []
    with torch.no_grad():
        for imgs, labels in val_loader:
            imgs = imgs.to(DEVICE)
            labels = labels.to(DEVICE)
            outputs = model(imgs)
            preds = outputs.argmax(dim=1)
            all_preds.extend(preds.cpu().numpy().tolist())
            all_labels.extend(labels.cpu().numpy().tolist())

    cm = confusion_matrix(all_labels, all_preds)
    plt.figure(figsize=(6,5))
    sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', xticklabels=class_names, yticklabels=class_names)
    plt.xlabel('Predicted'); plt.ylabel('True'); plt.title('Confusion Matrix')
    plt.show()

    print(classification_report(all_labels, all_preds, target_names=class_names))
else:
    print("Best model not found - no evaluation.")


## 9. Экспорт в ONNX и проверка через ONNX Runtime

Экспортируем лучшую модель в формат ONNX и проверим с помощью onnxruntime.

In [ ]:

import onnx
import torch.onnx
import onnxruntime as ort

best = OUT_DIR / "best_efficientnet_b0.pth"
onnx_out = OUT_DIR / "efficientnet_b0.onnx"

if best.exists():
    # создаём модель и dummy input
    model_cpu = timm.create_model(MODEL_NAME, pretrained=False, num_classes=len(class_names))
    model_cpu.load_state_dict(torch.load(best, map_location='cpu'))
    model_cpu.eval()

    dummy = torch.randn(1,3,IMAGE_SIZE,IMAGE_SIZE, device='cpu')
    torch.onnx.export(model_cpu, dummy, str(onnx_out), opset_version=18,
                      input_names=['input'], output_names=['output'], dynamic_axes={'input':{0:'batch_size'}, 'output':{0:'batch_size'}})
    print("ONNX exported to", onnx_out)

    # Проверка
    onnx_model = onnx.load(str(onnx_out))
    onnx.checker.check_model(onnx_model)
    print("ONNX model checked.")

    # Быстрая инференс-проверка через onnxruntime
    ort_sess = ort.InferenceSession(str(onnx_out), providers=['CPUExecutionProvider'])
    import numpy as np
    x = np.random.randn(1,3,IMAGE_SIZE,IMAGE_SIZE).astype(np.float32)
    out = ort_sess.run(None, {'input': x})
    print("ONNX runtime output shape:", np.array(out[0]).shape)
else:
    print("Best model weights not found, skipping ONNX export.")


## 10. Выводы

Кратко опишите результаты: точности, наблюдения (что модель путает), предложения по улучшению (больше данных, дополнительные аугментации, балансировка классов, изменение learning rate и/или полная разморозка backbone и дообучение).